In [33]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [34]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = LogisticRegressionCV(n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [35]:
def RF(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [54]:
factor = 0.8
nr_mean = 1
mean_tables = []
nr_samples=2000

datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
detectors = ['lid', 'multilid']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(nor), 2))
    
    results[it] = {}
    
    for dataset in datasets:
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                
            print("config: ", random_state, dataset, model)
            
            for detector in detectors:
                base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k20/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k20/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k20/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k20/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k20/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k20/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k20/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k20/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k20/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k20/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k20/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k20/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2: 
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))

                    if detector == 'lid':
                        y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                    elif detector == 'multilid':
                        y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    f1 =  round(100*f1_score(y_test, y_hat), 2)
                    
                    print(results)
     
                    if not detector in results[it][dataset][model]:
                        results[it][dataset][model][detector] = {}
                    
                    if not attacks[rows] in results[it][dataset][model][detector]: 
                        results[it][dataset][model][detector][attacks[rows]] = {}
                    
                    
                    if not 'auc' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['auc'] = {}
                    if not 'f1' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['f1' ] = {}
                        
                    results[it][dataset][model][detector][attacks[rows]]['auc'] = auc
                    results[it][dataset][model][detector][attacks[rows]]['f1'] = f1                    

                    print(attacks[rows].upper(), '&' , auc, '&', f1)

final_mean = np.mean(mean_tables, axis=0)
final_var  = np.var(mean_tables, axis=0)

random state:  21
config:  21 cifar10 wrn28-10
{0: {'cifar10': {'wrn28-10': {}}}}
FGSM & 99.0 & 93.02
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 99.0, 'f1': 93.02}}}}}}
BIM & 87.04 & 80.59
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 99.0, 'f1': 93.02}, 'bim': {'auc': 87.04, 'f1': 80.59}}}}}}
PGD & 88.34 & 78.73
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 99.0, 'f1': 93.02}, 'bim': {'auc': 87.04, 'f1': 80.59}, 'pgd': {'auc': 88.34, 'f1': 78.73}}}}}}
AA & 97.32 & 91.56
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 99.0, 'f1': 93.02}, 'bim': {'auc': 87.04, 'f1': 80.59}, 'pgd': {'auc': 88.34, 'f1': 78.73}, 'aa': {'auc': 97.32, 'f1': 91.56}}}}}}
DF & 95.53 & 87.95
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 99.0, 'f1': 93.02}, 'bim': {'auc': 87.04, 'f1': 80.59}, 'pgd': {'auc': 88.34, 'f1': 78.73}, 'aa': {'auc': 97.32, 'f1': 91.56}, 'df': {'auc': 95.53, 'f1': 87.95}}}}}}
CW & 94.5 & 85.35
{0: {'cifar10': {'wrn28-10': {'lid': {'fgsm': {'auc': 9

FileNotFoundError: [Errno 2] No such file or directory: '/home/lorenzp/workspace/multiLID/data/extract/run_1/cifar100/vgg16/multilid/pgd/k20/multilid_normalos_8255.pt'

In [45]:
final_mean

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])